In [ ]:
import numpy as np
import healpy as hp
from healpy import projview
import emcee
from tqdm import tqdm
import matplotlib.pyplot as plt
import pyccl as ccl
import pandas as pd
from astropy.coordinates import SkyCoord
import os

# ==============================================
# 1. LEITURA E PREPARAÇÃO DOS DADOS OBSERVACIONAIS
# ==============================================

# Parâmetros fixos
H0 = 67.27  # Valor fixo do Hubble
h = H0 / 100
Omega_b = 0.0494
sigma8 = 0.8120
n_s = 0.9646

# Carregar o catálogo Pantheon+
data = pd.read_csv('/home/sofia/Documentos/Pantheon/Pantheon+SH0ES.csv', sep=' ')
cove = np.genfromtxt('/home/sofia/Documentos/Pantheon/Pantheon+SH0ES_STAT+SYS.cov', skip_header=1)
full_cov = np.reshape(cove, (1701, 1701))

# Adicionar índice às supernovas
data['index'] = np.arange(len(data))

# Filtrar por redshift (exemplo: 0.1 < z < 1.0)
z_min, z_max = 0.15, 0.5
data_filtered = data[(data['zCMB'] >= z_min) & (data['zCMB'] <= z_max)]

In [ ]:
# ==============================================
# 2. DIVISÃO EM PIXELS HEALPix (nside=4)
# ==============================================

nside = 2 
npix = hp.nside2npix(nside)
raio = 90  # graus

# Obter coordenadas dos centros dos pixels
lon, lat = hp.pix2ang(nside, range(npix), lonlat=True)
galactic_coords = SkyCoord(l=lon, b=lat, frame='galactic', unit='deg')
equatorial_coords = galactic_coords.icrs
ra_pixel = equatorial_coords.ra.deg
dec_pixel = equatorial_coords.dec.deg

# Criar diretório para salvar os pixels
output_dir = '/home/sofia/Documentos/Pantheon/Atividade 4/Hemisferios_MCMC'  
os.makedirs(output_dir, exist_ok=True)
 

In [ ]:
# ==============================================
# 3. ASSOCIAR SUPERNOVAS A CADA PIXEL
# ==============================================

# Converter coordenadas das supernovas para radianos
ra_data = np.radians(data_filtered['RA'].values)
dec_data = np.radians(data_filtered['DEC'].values)

# Para cada pixel, encontrar supernovas dentro do raio
pixel_data_list = []
for i in tqdm(range(npix), desc="Processando pixels"):
    # Coordenadas do centro do pixel
    ra_centro = np.radians(ra_pixel[i])
    dec_centro = np.radians(dec_pixel[i])
    
    # Fórmula de Haversine
    delta_ra = ra_data - ra_centro
    delta_dec = dec_data - dec_centro
    a = np.sin(delta_dec/2)**2 + np.cos(dec_centro)*np.cos(dec_data)*np.sin(delta_ra/2)**2
    distancia = np.degrees(2 * np.arcsin(np.sqrt(a)))
    
    # Selecionar supernovas dentro do raio
    mask = distancia <= raio
    pixel_data = data_filtered[mask].copy()
    
    # Salvar dados do pixel
    pixel_file = os.path.join(output_dir, f'pixel_{i:03d}.csv')
    pixel_data.to_csv(pixel_file, index=False)
    pixel_data_list.append(pixel_data)

In [ ]:
# ==============================================
# 4. FUNÇÕES PARA O AJUSTE MCMC (MODIFICADAS)
# ==============================================

def mu_theory(z, Omega_m):
    """Calcula a magnitude de distância teórica usando PyCCL"""
    cosmo = ccl.Cosmology(
        Omega_c=Omega_m - Omega_b,
        Omega_b=Omega_b,
        h=h,
        sigma8=sigma8,
        n_s=n_s)
    a = 1/(1+z)
    return ccl.background.distance_modulus(cosmo, a)

def log_probability(theta, z_obs, mu_obs, inv_cov):
    """Função de log-probabilidade para o MCMC com prior mais amplo"""
    Omega_m = theta[0]
    
    # Prior ampliado para teste
    if not 0.01 < Omega_m < 1.0:
        return -np.inf
    
    # Calcular predição teórica
    mu_theo = mu_theory(z_obs, Omega_m)
    
    # Calcular chi²
    delta = mu_obs - mu_theo
    chi2 = np.dot(delta, np.dot(inv_cov, delta))
    
    return -0.5 * chi2

In [ ]:
# ==============================================
# 5. EXECUÇÃO DO MCMC MELHORADA PARA CADA PIXEL
# ==============================================

# Configuração do MCMC melhorada
nwalkers = 5   
nsteps = 500    
burnin = 100    

# Inicializar arrays para armazenar resultados
Om_map_mcmc = np.full(npix, np.nan)
Om_error_map = np.full(npix, np.nan)
acceptance_rates = np.full(npix, np.nan)

for i in tqdm(range(npix), desc="Ajustando pixels com MCMC"):
    try:
        pixel_file = os.path.join(output_dir, f'pixel_{i:03d}.csv')
        
        # Verificação rigorosa do arquivo
        if not os.path.exists(pixel_file) or os.path.getsize(pixel_file) == 0:
            continue
            
        pixel_data = pd.read_csv(pixel_file)
        
        # Verificação dos dados básicos
        if len(pixel_data) < 5:  # Requer pelo menos 5 supernovas
            continue
            
        z_obs = pixel_data['zCMB'].values
        mu_obs = pixel_data['MU_SH0ES'].values
        indices = pixel_data['index'].values
        
        print(f"\n=== Pixel {i} ===")
        print(f"> N_sne = {len(pixel_data)}")
        print(f"> z range = [{z_obs.min():.3f}, {z_obs.max():.3f}]")
        print(f"> mu range = [{mu_obs.min():.3f}, {mu_obs.max():.3f}]")
        
        # Manipulação robusta da matriz de covariância
        try:
            cov_sub = full_cov[np.ix_(indices, indices)]
            cond_number = np.linalg.cond(cov_sub)
            print(f"> Número de condicionamento: {cond_number:.2e}")
            
            if cond_number > 1e15:
                print("> Matriz mal condicionada - usando apenas diagonal")
                inv_cov_sub = np.diag(1/np.diag(cov_sub))
            else:
                inv_cov_sub = np.linalg.pinv(cov_sub)
        except Exception as e:
            print(f"> Erro na matriz de cov: {str(e)}")
            continue
            
        # Configuração inicial do MCMC
        initial_guess = 0.3 + 0.1*(i/npix)  # Pequena variação inicial por pixel
        pos = initial_guess + 0.05 * np.random.randn(nwalkers, 1)
        
        # Execução do MCMC
        sampler = emcee.EnsembleSampler(
            nwalkers, 1, log_probability,
            args=(z_obs, mu_obs, inv_cov_sub))
        
        # Rodar o MCMC com barra de progresso
        print("> Rodando MCMC...")
        sampler.run_mcmc(pos, nsteps, progress=True)
        
        # Análise das cadeias
        samples = sampler.get_chain(discard=burnin, flat=True)
        
        if len(samples) == 0:
            continue
            
        # Cálculo das estatísticas
        Om_map_mcmc[i] = np.median(samples)
        Om_error_map[i] = np.std(samples)
        acceptance_rates[i] = np.mean(sampler.acceptance_fraction)
        
        print(f"> Resultado: Ωm = {Om_map_mcmc[i]:.3f} ± {Om_error_map[i]:.3f}")
        print(f"> Taxa de aceitação: {acceptance_rates[i]:.2f}")
        
        # Diagnóstico avançado
        try:
            tau = sampler.get_autocorr_time()
            print(f"> Tempo de autocorrelação: {tau[0]:.2f}")
        except:
            print("> Não foi possível calcular autocorrelação")
        
        # Plot de diagnóstico melhorado
        plt.figure(figsize=(12, 5))
        
        plt.subplot(1, 2, 1)
        plt.plot(sampler.get_chain()[:, :, 0].T, alpha=0.4)
        plt.title(f"Pixel {i} - Cadeias MCMC")
        plt.ylabel("Ωm")
        
        plt.subplot(1, 2, 2)
        plt.hist(samples, bins=30, density=True)
        plt.title(f"Distribuição posterior")
        plt.xlabel("Ωm")
        
        plt.tight_layout()
        plt.savefig(os.path.join(output_dir, f'diagnostics_pixel_{i:03d}.png'))
        plt.close()
        
    except Exception as e:
        print(f"> ERRO no pixel {i}: {str(e)}")
        continue

In [ ]:
# ==============================================
# 6. SALVAR E VISUALIZAR RESULTADOS FINAIS
# ==============================================

# Salvar resultados em um arquivo
results = pd.DataFrame({
    'pixel': np.arange(npix),
    'ra_center': ra_pixel,
    'dec_center': dec_pixel,
    'Omega_m': Om_map_mcmc,
    'Omega_m_error': Om_error_map,
    'acceptance_rate': acceptance_rates,
    'n_sne': [len(pd.read_csv(os.path.join(output_dir, f'pixel_{i:03d}.csv'))) for i in range(npix)]
})

results.to_csv(os.path.join(output_dir, 'mcmc_results.csv'), index=False)

# Visualização dos resultados em um mapa
plt.figure(figsize=(10, 7))
hp.mollview(Om_map_mcmc, title="Ωm por pixel", unit="", min=0.2, max=0.6)
plt.savefig(os.path.join(output_dir, 'omega_m_map.png'))
plt.close()

print("\n=== ANÁLISE FINAL ===")
print(f"Média de Ωm: {np.nanmean(Om_map_mcmc):.3f} ± {np.nanstd(Om_map_mcmc):.3f}")
print(f"Taxa de aceitação média: {np.nanmean(acceptance_rates):.2f}")